In [1]:
apiKey = 'pk.eyJ1Ijoia2FtZW5ldHNraXl0byIsImEiOiJjbGEyc29sa3AwamczM25waHJiNGVoY2kxIn0.ptlM5x8rtFOVxWgK9ZT2oQ'

In [2]:
import pandas as pd
import requests

In [3]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import json

In [4]:
import psycopg2 as pg
import pandas.io.sql as psql
connection = pg.connect("host=188.72.109.162 dbname=optimus_post_db user=reader password=aboba")
cursor = connection.cursor()

In [5]:
zamDf = pd.read_csv('data/zamosk.csv')
zamDf = zamDf.drop([47, 78, 86, 102, 105, 109, 136])
zamOstDf = pd.read_csv('data/zamosk_ost.csv')
housesNew = pd.read_csv('data/houses_new.csv')

In [6]:
housesNew.shape

(33398, 7)

In [39]:
zamOstDf

,Unnamed: 0,id,global_id,geo_data,mos_dataset_id,data,area_id,coordinates
0,680,1009078,833046185,"{'type': 'Point', 'coordinates': [37.62770898,...",60662,"{'Street': '', 'geoData': {'type': 'Point', 'c...",30.0,"[37.62770898, 55.72210574]"
1,681,1009079,833046186,"{'type': 'Point', 'coordinates': [37.62622664,...",60662,"{'Street': '', 'geoData': {'type': 'Point', 'c...",30.0,"[37.62622664, 55.72499391]"
2,688,1009086,833046193,"{'type': 'Point', 'coordinates': [37.62219435,...",60662,"{'Street': '', 'geoData': {'type': 'Point', 'c...",30.0,"[37.62219435, 55.72263741]"
3,1508,1009908,833047412,"{'type': 'Point', 'coordinates': [37.62496846,...",60662,"{'Street': '', 'geoData': {'type': 'Point', 'c...",30.0,"[37.62496846, 55.72959896]"
4,1786,1010186,833047786,"{'type': 'Point', 'coordinates': [37.63462863,...",60662,"{'Street': '', 'geoData': {'type': 'Point', 'c...",30.0,"[37.63462863, 55.74430251]"
5,1791,1010191,833047791,"{'type': 'Point', 'coordinates': [37.63439908,...",60662,"{'Street': '', 'geoData': {'type': 'Point', 'c...",30.0,"[37.63439908, 55.74443474]"
6,1792,1010192,833047792,"{'type': 'Point', 'coordinates': [37.63027594,...",60662,"{'Street': '', 'geoData': {'type': 'Point', 'c...",30.0,"[37.63027594, 55.74257538]"
7,1833,1010233,833047845,"{'type': 'Point', 'coordinates': [37.62801015,...",60662,"{'Street': '', 'geoData': {'type': 'Point', 'c...",30.0,"[37.62801015, 55.73560986]"
8,1834,1010234,833047846,"{'type': 'Point', 'coordinates': [37.62851694,...",60662,"{'Street': '', 'geoData': {'type': 'Point', 'c...",30.0,"[37.62851694, 55.7392087]"
9,1835,1010235,833047847,"{'type': 'Point', 'coordinates': [37.62863125,...",60662,"{'Street': '', 'geoData': {'type': 'Point', 'c...",30.0,"[37.62863125, 55.74230524]"


In [7]:
import ast
zamOstDf['coordinates'] = zamOstDf['coordinates'].apply(ast.literal_eval)

In [8]:
zamDf['coordinates'] = zamDf['coordinates'].apply(ast.literal_eval)

In [16]:
routes = pd.DataFrame(columns=['house_id', 'ost_id', 'route', 'duration', 'distance'])
index = 0
for index, row in zamDf.iterrows():
    if index < 0:
        index += 1
    else:
        for index2, row2 in zamOstDf.iterrows():
            coordinates = str(row['coordinates'][0]) + ',' + str(row['coordinates'][1]) + ';' + str(row2['coordinates'][0]) + ',' + str(row2['coordinates'][1])
            url = 'https://api.mapbox.com/directions/v5/mapbox/walking/' + coordinates + '?access_token=' + apiKey
            r = requests.get(url, params={'alternatives': 'true', 'geometries': 'geojson'})
            routes.loc[len(routes)] = [row['id'], row2['id'], r.json()['routes'][0]['geometry'], r.json()['routes'][0]['duration'], r.json()['routes'][0]['distance']]
        index += 1
        print(index)
        if index % 4 == 0:
            time.sleep(60)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
80
81
82
83
84
85
86
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
104
105
107
108
109
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
138


In [29]:
metroVokzalDf = pd.DataFrame(columns=['id', 'name', 'coordinates', 'area_id'])

In [30]:
metroVokzalDf.loc[len(metroVokzalDf)] = [0, 'Павелецкий Вокзал', ['37.640900', '55.730129'], '30']
metroVokzalDf.loc[len(metroVokzalDf)] = [1, 'Метро Третьяковская', ['37.625700', '55.741151'], '30']
metroVokzalDf.loc[len(metroVokzalDf)] = [2, 'Метро Новокузнецкая', ['37.629318', '55.742346'], '30']
metroVokzalDf.loc[len(metroVokzalDf)] = [3, 'Метро Павелецкая', ['37.637150', '55.731598'], '30']
metroVokzalDf.loc[len(metroVokzalDf)] = [4, 'Метро Серпуховская', ['37.622855', '55.728797'], '30']
metroVokzalDf.loc[len(metroVokzalDf)] = [5, 'Метро Добрынинская', ['37.625061', '55.727253'], '30']

In [34]:
routesMetroVokzal = pd.DataFrame(columns=['house_id', 'met_vok_id', 'route', 'duration', 'distance'])
index = 0
for index, row in zamDf.iterrows():
    if index < 0:
        index += 1
    else:
        for index2, row2 in metroVokzalDf.iterrows():
            coordinates = str(row['coordinates'][0]) + ',' + str(row['coordinates'][1]) + ';' + str(row2['coordinates'][0]) + ',' + str(row2['coordinates'][1])
            url = 'https://api.mapbox.com/directions/v5/mapbox/walking/' + coordinates + '?access_token=' + apiKey
            r = requests.get(url, params={'alternatives': 'true', 'geometries': 'geojson'})
            routesMetroVokzal.loc[len(routesMetroVokzal)] = [row['id'], row2['id'], r.json()['routes'][0]['geometry'], r.json()['routes'][0]['duration'], r.json()['routes'][0]['distance']]
        index += 1
        print(index)
        if index % 4 == 0:
            time.sleep(10)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
80
81
82
83
84
85
86
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
104
105
107
108
109
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
138


In [36]:
routes.to_csv('data/routes_bus_zamosk.csv')
routesMetroVokzal.to_csv('data/routes_metro_train_zamosk.csv')

In [43]:
def get_type(string):
    if string == 0:
        return 1
    else:
        return 2

In [ ]:
routes['transport_type'] = 0


In [38]:
routes

,house_id,ost_id,route,duration,distance
0,2021,1009078,"{'coordinates': [[37.617231, 55.72235], [37.61...",691.977,937.778
1,2021,1009079,"{'coordinates': [[37.617231, 55.72235], [37.61...",750.756,1042.037
2,2021,1009086,"{'coordinates': [[37.617231, 55.72235], [37.61...",413.572,568.107
3,2021,1009908,"{'coordinates': [[37.617231, 55.72235], [37.61...",857.254,1225.691
4,2021,1010186,"{'coordinates': [[37.617231, 55.72235], [37.61...",2241.826,3157.877
...,...,...,...,...,...
7593,32586,1020263,"{'coordinates': [[37.629329, 55.73528], [37.62...",1153.194,1615.894
7594,32586,1020265,"{'coordinates': [[37.629329, 55.73528], [37.62...",1243.655,1739.277
7595,32586,1020271,"{'coordinates': [[37.629329, 55.73528], [37.62...",1141.092,1603.932
7596,32586,1020295,"{'coordinates': [[37.629329, 55.73528], [37.62...",1268.886,1732.584


In [44]:
routesMetroVokzal['transport_type'] = routesMetroVokzal['met_vok_id'].apply(get_type)

In [45]:
routesMetroVokzal

,house_id,met_vok_id,route,duration,distance,transport_type
0,2021,0,"{'coordinates': [[37.617231, 55.72235], [37.61...",1541.943,2173.324,1
1,2021,1,"{'coordinates': [[37.617231, 55.72235], [37.61...",1846.710,2606.553,2
2,2021,2,"{'coordinates': [[37.617231, 55.72235], [37.61...",1975.561,2778.720,2
3,2021,3,"{'coordinates': [[37.617231, 55.72235], [37.61...",1455.051,2086.276,2
4,2021,4,"{'coordinates': [[37.617231, 55.72235], [37.61...",717.753,1022.795,2
...,...,...,...,...,...,...
781,32586,1,"{'coordinates': [[37.629329, 55.73528], [37.62...",778.803,1068.485,2
782,32586,2,"{'coordinates': [[37.629329, 55.73528], [37.62...",734.090,994.697,2
783,32586,3,"{'coordinates': [[37.629329, 55.73528], [37.62...",678.496,933.058,2
784,32586,4,"{'coordinates': [[37.629329, 55.73528], [37.62...",757.183,994.419,2


In [47]:
routesMetroVokzal.loc[routesMetroVokzal['transport_type'] == 2]

,house_id,met_vok_id,route,duration,distance,transport_type
1,2021,1,"{'coordinates': [[37.617231, 55.72235], [37.61...",1846.710,2606.553,2
2,2021,2,"{'coordinates': [[37.617231, 55.72235], [37.61...",1975.561,2778.720,2
3,2021,3,"{'coordinates': [[37.617231, 55.72235], [37.61...",1455.051,2086.276,2
4,2021,4,"{'coordinates': [[37.617231, 55.72235], [37.61...",717.753,1022.795,2
5,2021,5,"{'coordinates': [[37.617231, 55.72235], [37.61...",735.020,1042.486,2
...,...,...,...,...,...,...
781,32586,1,"{'coordinates': [[37.629329, 55.73528], [37.62...",778.803,1068.485,2
782,32586,2,"{'coordinates': [[37.629329, 55.73528], [37.62...",734.090,994.697,2
783,32586,3,"{'coordinates': [[37.629329, 55.73528], [37.62...",678.496,933.058,2
784,32586,4,"{'coordinates': [[37.629329, 55.73528], [37.62...",757.183,994.419,2


In [48]:
routesMetroVokzal['wtf'] = routesMetroVokzal['distance'] / routesMetroVokzal['duration']

In [50]:
routes

,house_id,ost_id,route,duration,distance
0,2021,1009078,"{'coordinates': [[37.617231, 55.72235], [37.61...",691.977,937.778
1,2021,1009079,"{'coordinates': [[37.617231, 55.72235], [37.61...",750.756,1042.037
2,2021,1009086,"{'coordinates': [[37.617231, 55.72235], [37.61...",413.572,568.107
3,2021,1009908,"{'coordinates': [[37.617231, 55.72235], [37.61...",857.254,1225.691
4,2021,1010186,"{'coordinates': [[37.617231, 55.72235], [37.61...",2241.826,3157.877
...,...,...,...,...,...
7593,32586,1020263,"{'coordinates': [[37.629329, 55.73528], [37.62...",1153.194,1615.894
7594,32586,1020265,"{'coordinates': [[37.629329, 55.73528], [37.62...",1243.655,1739.277
7595,32586,1020271,"{'coordinates': [[37.629329, 55.73528], [37.62...",1141.092,1603.932
7596,32586,1020295,"{'coordinates': [[37.629329, 55.73528], [37.62...",1268.886,1732.584


In [59]:
routes['distance'][:3].sort_values()

2     568.107
0     937.778
1    1042.037
Name: distance, dtype: float64

In [58]:
routes['distance'][:3].sort_values().index

Int64Index([2, 0, 1], dtype='int64')

In [61]:
routesMetroVokzal

,house_id,met_vok_id,route,duration,distance,transport_type,wtf
0,2021,0,"{'coordinates': [[37.617231, 55.72235], [37.61...",1541.943,2173.324,1,1.409471
1,2021,1,"{'coordinates': [[37.617231, 55.72235], [37.61...",1846.710,2606.553,2,1.411458
2,2021,2,"{'coordinates': [[37.617231, 55.72235], [37.61...",1975.561,2778.720,2,1.406547
3,2021,3,"{'coordinates': [[37.617231, 55.72235], [37.61...",1455.051,2086.276,2,1.433816
4,2021,4,"{'coordinates': [[37.617231, 55.72235], [37.61...",717.753,1022.795,2,1.424996
...,...,...,...,...,...,...,...
781,32586,1,"{'coordinates': [[37.629329, 55.73528], [37.62...",778.803,1068.485,2,1.371958
782,32586,2,"{'coordinates': [[37.629329, 55.73528], [37.62...",734.090,994.697,2,1.355007
783,32586,3,"{'coordinates': [[37.629329, 55.73528], [37.62...",678.496,933.058,2,1.375186
784,32586,4,"{'coordinates': [[37.629329, 55.73528], [37.62...",757.183,994.419,2,1.313314


In [63]:
routesMetroVokzalDubl = pd.DataFrame(columns=['house_id', 'met_vok_id', 'route', 'duration', 'distance'])
routesMetroVokzalDubl = routesMetroVokzalDubl.append(routesMetroVokzal.loc[0])
routesMetroVokzalDubl

,house_id,met_vok_id,route,duration,distance,transport_type,wtf
0,2021,0,"{'coordinates': [[37.617231, 55.72235], [37.61...",1541.943,2173.324,1.0,1.409471


In [85]:
routes['distance'][:3].sort_values().to_list()[0]
routes['distance'][index:index + 3].sort_values().to_list()[0]

385.931

In [98]:
routes[7500:]

,house_id,ost_id,route,duration,distance
7500,19975,1012703,"{'coordinates': [[37.630568, 55.736472], [37.6...",519.329,678.133
7501,19975,1012705,"{'coordinates': [[37.630568, 55.736472], [37.6...",808.301,1029.894
7502,19975,1012707,"{'coordinates': [[37.630568, 55.736472], [37.6...",1022.181,1371.170
7503,19975,1012708,"{'coordinates': [[37.630568, 55.736472], [37.6...",1331.219,1801.367
7504,19975,1012711,"{'coordinates': [[37.630568, 55.736472], [37.6...",1152.969,1540.092
...,...,...,...,...,...
7593,32586,1020263,"{'coordinates': [[37.629329, 55.73528], [37.62...",1153.194,1615.894
7594,32586,1020265,"{'coordinates': [[37.629329, 55.73528], [37.62...",1243.655,1739.277
7595,32586,1020271,"{'coordinates': [[37.629329, 55.73528], [37.62...",1141.092,1603.932
7596,32586,1020295,"{'coordinates': [[37.629329, 55.73528], [37.62...",1268.886,1732.584


In [95]:
routesDubl = pd.DataFrame(columns=['house_id', 'met_vok_id', 'route', 'duration', 'distance'])
index1 = routes['distance'][:3].sort_values().index[0]
index2 = routes['distance'][:3].sort_values().index[1]
index3 = routes['distance'][:3].sort_values().index[2]
high1 = routes['distance'][:3].sort_values().to_list()[0]
high2 = routes['distance'][:3].sort_values().to_list()[1]
high3 = routes['distance'][:3].sort_values().to_list()[2]

current = routes.loc[0]['house_id']
i = 0
for index, row in routes.iterrows():
    if row['house_id'] != current:
        routesDubl = routesDubl.append(routes.loc[index1])
        routesDubl = routesDubl.append(routes.loc[index2])
        routesDubl = routesDubl.append(routes.loc[index3])
        index1 = routes['distance'][index:index + 3].sort_values().index[0]
        index2 = routes['distance'][index:index + 3].sort_values().index[1]
        index3 = routes['distance'][index:index + 3].sort_values().index[2]
        high1 = routes['distance'][index:index + 3].sort_values().to_list()[0]
        high2 = routes['distance'][index:index + 3].sort_values().to_list()[1]
        high3 = routes['distance'][index:index + 3].sort_values().to_list()[2]
        current = row['house_id']
    else:
        if row['distance'] < high1:
            high3 = high2
            high2 = high1
            high1 = row['distance']
        elif row['distance'] < high2:
            high3 = high2
            high2 = row['distance']
        elif row['distance'] < high3:
            high3 = row['distance']
    if i % 10 == 0:
        print(i)
    i += 1

routesDubl = routesDubl.append(routes.loc[index1])
routesDubl = routesDubl.append(routes.loc[index2])
routesDubl = routesDubl.append(routes.loc[index3])

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2

In [96]:
routesDubl

,house_id,met_vok_id,route,duration,distance,ost_id
2,2021,NaN,"{'coordinates': [[37.617231, 55.72235], [37.61...",413.572,568.107,1009086.0
0,2021,NaN,"{'coordinates': [[37.617231, 55.72235], [37.61...",691.977,937.778,1009078.0
1,2021,NaN,"{'coordinates': [[37.617231, 55.72235], [37.61...",750.756,1042.037,1009079.0
58,2556,NaN,"{'coordinates': [[37.632089, 55.723585], [37.6...",334.261,385.931,1009078.0
59,2556,NaN,"{'coordinates': [[37.632089, 55.723585], [37.6...",460.902,617.249,1009079.0
...,...,...,...,...,...,...
7482,19975,NaN,"{'coordinates': [[37.630568, 55.736472], [37.6...",1314.154,1784.612,1009078.0
7484,19975,NaN,"{'coordinates': [[37.630568, 55.736472], [37.6...",1379.544,1837.683,1009086.0
7541,32586,NaN,"{'coordinates': [[37.629329, 55.73528], [37.62...",953.842,1287.044,1009079.0
7540,32586,NaN,"{'coordinates': [[37.629329, 55.73528], [37.62...",1192.866,1621.962,1009078.0


In [102]:
def get_route(string):
    return string['coordinates']

In [243]:
routesMetroVokzal['coords'] = routesMetroVokzal['route'].apply(get_route)

In [103]:
routesDubl['coords'] = routesDubl['route'].apply(get_route)

In [106]:
routesDubl

,house_id,met_vok_id,route,duration,distance,ost_id,coords
2,2021,NaN,"{'coordinates': [[37.617231, 55.72235], [37.61...",413.572,568.107,1009086.0,"[[37.617231, 55.72235], [37.616985, 55.723148]..."
0,2021,NaN,"{'coordinates': [[37.617231, 55.72235], [37.61...",691.977,937.778,1009078.0,"[[37.617231, 55.72235], [37.61767, 55.720968],..."
1,2021,NaN,"{'coordinates': [[37.617231, 55.72235], [37.61...",750.756,1042.037,1009079.0,"[[37.617231, 55.72235], [37.616985, 55.723148]..."
58,2556,NaN,"{'coordinates': [[37.632089, 55.723585], [37.6...",334.261,385.931,1009078.0,"[[37.632089, 55.723585], [37.632121, 55.723556..."
59,2556,NaN,"{'coordinates': [[37.632089, 55.723585], [37.6...",460.902,617.249,1009079.0,"[[37.632089, 55.723585], [37.632121, 55.723556..."
...,...,...,...,...,...,...,...
7482,19975,NaN,"{'coordinates': [[37.630568, 55.736472], [37.6...",1314.154,1784.612,1009078.0,"[[37.630568, 55.736472], [37.630683, 55.736495..."
7484,19975,NaN,"{'coordinates': [[37.630568, 55.736472], [37.6...",1379.544,1837.683,1009086.0,"[[37.630568, 55.736472], [37.630683, 55.736495..."
7541,32586,NaN,"{'coordinates': [[37.629329, 55.73528], [37.62...",953.842,1287.044,1009079.0,"[[37.629329, 55.73528], [37.62862, 55.735103],..."
7540,32586,NaN,"{'coordinates': [[37.629329, 55.73528], [37.62...",1192.866,1621.962,1009078.0,"[[37.629329, 55.73528], [37.62862, 55.735103],..."


In [242]:
routesDubl

,house_id,met_vok_id,route,duration,distance,ost_id,coords
2,2021,NaN,"{'coordinates': [[37.617231, 55.72235], [37.61...",413.572,568.107,1009086.0,"[[37.617231, 55.72235], [37.616985, 55.723148]..."
0,2021,NaN,"{'coordinates': [[37.617231, 55.72235], [37.61...",691.977,937.778,1009078.0,"[[37.617231, 55.72235], [37.61767, 55.720968],..."
1,2021,NaN,"{'coordinates': [[37.617231, 55.72235], [37.61...",750.756,1042.037,1009079.0,"[[37.617231, 55.72235], [37.616985, 55.723148]..."
58,2556,NaN,"{'coordinates': [[37.632089, 55.723585], [37.6...",334.261,385.931,1009078.0,"[[37.632089, 55.723585], [37.632121, 55.723556..."
59,2556,NaN,"{'coordinates': [[37.632089, 55.723585], [37.6...",460.902,617.249,1009079.0,"[[37.632089, 55.723585], [37.632121, 55.723556..."
...,...,...,...,...,...,...,...
7482,19975,NaN,"{'coordinates': [[37.630568, 55.736472], [37.6...",1314.154,1784.612,1009078.0,"[[37.630568, 55.736472], [37.630683, 55.736495..."
7484,19975,NaN,"{'coordinates': [[37.630568, 55.736472], [37.6...",1379.544,1837.683,1009086.0,"[[37.630568, 55.736472], [37.630683, 55.736495..."
7541,32586,NaN,"{'coordinates': [[37.629329, 55.73528], [37.62...",953.842,1287.044,1009079.0,"[[37.629329, 55.73528], [37.62862, 55.735103],..."
7540,32586,NaN,"{'coordinates': [[37.629329, 55.73528], [37.62...",1192.866,1621.962,1009078.0,"[[37.629329, 55.73528], [37.62862, 55.735103],..."


In [244]:
routesMetroVokzal

,house_id,met_vok_id,route,duration,distance,transport_type,wtf,coords
0,2021,0,"{'coordinates': [[37.617231, 55.72235], [37.61...",1541.943,2173.324,1,1.409471,"[[37.617231, 55.72235], [37.616985, 55.723148]..."
1,2021,1,"{'coordinates': [[37.617231, 55.72235], [37.61...",1846.710,2606.553,2,1.411458,"[[37.617231, 55.72235], [37.616985, 55.723148]..."
2,2021,2,"{'coordinates': [[37.617231, 55.72235], [37.61...",1975.561,2778.720,2,1.406547,"[[37.617231, 55.72235], [37.616985, 55.723148]..."
3,2021,3,"{'coordinates': [[37.617231, 55.72235], [37.61...",1455.051,2086.276,2,1.433816,"[[37.617231, 55.72235], [37.616985, 55.723148]..."
4,2021,4,"{'coordinates': [[37.617231, 55.72235], [37.61...",717.753,1022.795,2,1.424996,"[[37.617231, 55.72235], [37.616985, 55.723148]..."
...,...,...,...,...,...,...,...,...
781,32586,1,"{'coordinates': [[37.629329, 55.73528], [37.62...",778.803,1068.485,2,1.371958,"[[37.629329, 55.73528], [37.62862, 55.735103],..."
782,32586,2,"{'coordinates': [[37.629329, 55.73528], [37.62...",734.090,994.697,2,1.355007,"[[37.629329, 55.73528], [37.62862, 55.735103],..."
783,32586,3,"{'coordinates': [[37.629329, 55.73528], [37.62...",678.496,933.058,2,1.375186,"[[37.629329, 55.73528], [37.62862, 55.735103],..."
784,32586,4,"{'coordinates': [[37.629329, 55.73528], [37.62...",757.183,994.419,2,1.313314,"[[37.629329, 55.73528], [37.62862, 55.735103],..."


In [303]:
index = 0
all_routes = []
for route in routesDubl['coords']:
    single_route = []
    for coord in route:
        single_route.append(coord)
        if len(single_route) == 2:
            all_routes.append(single_route)
            single_route = []
            single_route.append(coord)
    index += 1

In [301]:
routesDubl['coords'] = routesDubl['route'].apply(get_route)
index = 0
all_routes = []
for route in routesDubl['coords']:
    single_route = []
    for coord in route:
        single_route.append(coord)
        if len(single_route) == 2:
            all_routes.append(str(single_route).replace('[', '').replace(']', ''))
            single_route = []
            single_route.append(coord)
    index += 1
c = Counter(all_routes)
c.most_common()
ret = pd.DataFrame(columns=['line'])
for i in range(0, 20):
    lili = []
    li = []
    li.append(li_from_string(c.most_common()[i][0]))
    lili.append(li)
    ret.loc[len(ret)] = lili
ret.to_csv('data/line_for_bus.csv')

In [296]:
c = Counter(all_routes)

In [297]:
c.most_common()

[('37.625342, 55.726962, 37.625505, 55.726659', 152),
 ('37.626555, 55.731733, 37.625649, 55.730064', 144),
 ('37.625206, 55.727647, 37.625342, 55.726962', 130),
 ('37.622251, 55.723947, 37.622203, 55.722637', 113),
 ('37.622673, 55.723922, 37.622251, 55.723947', 112),
 ('37.625803, 55.72961, 37.625206, 55.727647', 107),
 ('37.627292, 55.732519, 37.626555, 55.731733', 102),
 ('37.625505, 55.726659, 37.627742, 55.72211', 74),
 ('37.623808, 55.729219, 37.622673, 55.723922', 73),
 ('37.625375, 55.730082, 37.624114, 55.730023', 66),
 ('37.624114, 55.730023, 37.624165, 55.729739', 66),
 ('37.625649, 55.730064, 37.625375, 55.730082', 65),
 ('37.625649, 55.730064, 37.625803, 55.72961', 62),
 ('37.625505, 55.726659, 37.626241, 55.724996', 56),
 ('37.629641, 55.73052, 37.629803, 55.730415', 44),
 ('37.629803, 55.730415, 37.626749, 55.730126', 44),
 ('37.626749, 55.730126, 37.626724, 55.7302', 44),
 ('37.626724, 55.7302, 37.625681, 55.730124', 44),
 ('37.624165, 55.729739, 37.623889, 55.729608',

In [294]:
c.most_common()

[('37.625664, 55.740775, 37.625659, 55.740638', 131),
 ('37.625048, 55.740608, 37.625051, 55.740978', 131),
 ('37.625051, 55.740978, 37.625142, 55.741124', 131),
 ('37.625142, 55.741124, 37.625692, 55.741177', 131),
 ('37.629326, 55.742457, 37.629323, 55.742415', 131),
 ('37.629323, 55.742415, 37.629317, 55.742346', 131),
 ('37.625332, 55.726762, 37.62517, 55.727206', 131),
 ('37.625921, 55.726808, 37.625912, 55.726832', 114),
 ('37.625912, 55.726832, 37.625861, 55.726826', 114),
 ('37.625861, 55.726826, 37.62563, 55.726801', 114),
 ('37.62563, 55.726801, 37.625332, 55.726762', 114),
 ('37.625659, 55.740638, 37.625048, 55.740608', 113),
 ('37.623574, 55.728848, 37.62297, 55.728847', 113),
 ('37.62297, 55.728847, 37.62295, 55.728787', 113),
 ('37.626555, 55.731733, 37.625649, 55.730064', 91),
 ('37.625483, 55.726762, 37.625921, 55.726808', 81),
 ('37.629237, 55.742202, 37.629246, 55.742269', 75),
 ('37.625663, 55.740903, 37.625664, 55.740775', 74),
 ('37.627292, 55.732519, 37.626555, 55

In [315]:
def li_from_string(string):
    tmp = string[string.find(',') + 1:]
    tmp2 = string[:string.find(',')]
    tmpp1 = tmp[tmp.find(','):]
    tmpp2 = tmp[:tmp.find(',')]
    return '[' + tmp2 + ',' + tmpp2[1:] + ',' + tmpp1[2:].replace(' ', '') + ']'

In [277]:
li_from_string(c.most_common()[0][0])

'[37.625664,55.740775],[37.625659,55.740638]'

In [254]:
c.most_common()[0][0]

'37.625664, 55.740775, 37.625659, 55.740638'

In [316]:
ret = pd.DataFrame(columns=['line'])
for i in range(0, 20):
    lili = []
    li = []
    li.append(li_from_string(c.most_common()[i][0]))
    lili.append(li)
    print(li[0])

[37.625664,55.740775,37.625659,55.740638],
[37.625048,55.740608,37.625051,55.740978],
[37.625051,55.740978,37.625142,55.741124],
[37.625142,55.741124,37.625692,55.741177],
[37.629326,55.742457,37.629323,55.742415],
[37.629323,55.742415,37.629317,55.742346],
[37.625332,55.726762,37.62517,55.727206],
[37.625921,55.726808,37.625912,55.726832],
[37.625912,55.726832,37.625861,55.726826],
[37.625861,55.726826,37.62563,55.726801],
[37.62563,55.726801,37.625332,55.726762],
[37.625659,55.740638,37.625048,55.740608],
[37.623574,55.728848,37.62297,55.728847],
[37.62297,55.728847,37.62295,55.728787],
[37.626555,55.731733,37.625649,55.730064],
[37.625483,55.726762,37.625921,55.726808],
[37.629237,55.742202,37.629246,55.742269],
[37.625663,55.740903,37.625664,55.740775],
[37.627292,55.732519,37.626555,55.731733],
[37.625342,55.726962,37.625483,55.726762],


In [ ]:
for i in range(0, 20):

In [317]:
ret

,line


In [286]:
ret.to_csv('data/line_for_bus_with_count.csv')

[
[
[
[
[
[
[
[
[
[
[
[
[
[
[
[
[
[
[
[


In [319]:
tmp = pd.read_csv('data/line_for_bus.csv')

In [283]:
ret

,line
0,"[[37.625664,55.740775],[37.625659,55.740638]]"
1,"[[37.625048,55.740608],[37.625051,55.740978]]"
2,"[[37.625051,55.740978],[37.625142,55.741124]]"
3,"[[37.625142,55.741124],[37.625692,55.741177]]"
4,"[[37.629326,55.742457],[37.629323,55.742415]]"
5,"[[37.629323,55.742415],[37.629317,55.742346]]"
6,"[[37.625332,55.726762],[37.62517,55.727206]]"
7,"[[37.625921,55.726808],[37.625912,55.726832]]"
8,"[[37.625912,55.726832],[37.625861,55.726826]]"
9,"[[37.625861,55.726826],[37.62563,55.726801]]"


In [198]:
c = Counter()
for xs in all_routes:
    print(xs)
    for x in set(xs):
        c[x] += 1


[37.617231, 37.616985]
37.616985
37.617231
[37.616985, 37.617652]
37.617652
37.616985
[37.617652, 37.617593]
37.617593
37.617652
[37.617593, 37.621781]
37.621781
37.617593
[37.621781, 37.621735]
37.621735
37.621781
[37.621735, 37.622208]
37.622208
37.621735
[37.622208, 37.622203]
37.622203
37.622208
[37.617231, 37.61767]
37.61767
37.617231
[37.61767, 37.61959]
37.61959
37.61767
[37.61959, 37.619634]
37.619634
37.61959
[37.619634, 37.620018]
37.620018
37.619634
[37.620018, 37.620199]
37.620199
37.620018
[37.620199, 37.622527]
37.622527
37.620199
[37.622527, 37.622698]
37.622698
37.622527
[37.622698, 37.627108]
37.627108
37.622698
[37.627108, 37.62722]
37.62722
37.627108
[37.62722, 37.627874]
37.627874
37.62722
[37.627874, 37.627869]
37.627869
37.627874
[37.627869, 37.627742]
37.627742
37.627869
[37.617231, 37.616985]
37.616985
37.617231
[37.616985, 37.617652]
37.617652
37.616985
[37.617652, 37.617593]
37.617593
37.617652
[37.617593, 37.62222]
37.62222
37.617593
[37.62222, 37.622251]
37.

37.625681
[37.625649, 37.625375]
37.625375
37.625649
[37.625375, 37.624114]
37.624114
37.625375
[37.624114, 37.624165]
37.624165
37.624114
[37.624165, 37.623889]
37.623889
37.624165
[37.623889, 37.623808]
37.623808
37.623889
[37.623808, 37.622673]
37.622673
37.623808
[37.622673, 37.622251]
37.622251
37.622673
[37.622251, 37.622203]
37.622203
37.622251
[37.634246, 37.630465]
37.630465
37.634246
[37.630465, 37.627846]
37.627846
37.630465
[37.627846, 37.627292]
37.627292
37.627846
[37.627292, 37.626555]
37.626555
37.627292
[37.626555, 37.625649]
37.625649
37.626555
[37.625649, 37.625803]
37.625803
37.625649
[37.625803, 37.625206]
37.625206
37.625803
[37.625206, 37.625342]
37.625342
37.625206
[37.625342, 37.625505]
37.625505
37.625342
[37.625505, 37.626241]
37.626241
37.625505
[37.634246, 37.630465]
37.630465
37.634246
[37.630465, 37.627846]
37.627846
37.630465
[37.627846, 37.627292]
37.627292
37.627846
[37.627292, 37.626555]
37.626555
37.627292
[37.626555, 37.625649]
37.625649
37.626555
[

37.622343
[37.622173, 37.622306]
37.622306
37.622173
[37.622306, 37.622226]
37.622226
37.622306
[37.622226, 37.622443]
37.622443
37.622226
[37.622443, 37.62222]
37.62222
37.622443
[37.62222, 37.622203]
37.622203
37.62222
[37.622578, 37.622445]
37.622445
37.622578
[37.622445, 37.622804]
37.622804
37.622445
[37.622804, 37.622742]
37.622742
37.622804
[37.622742, 37.623037]
37.623037
37.622742
[37.623037, 37.623045]
37.623045
37.623037
[37.623045, 37.625184]
37.625184
37.623045
[37.625184, 37.625286]
37.625286
37.625184
[37.625286, 37.625631]
37.625631
37.625286
[37.625631, 37.627742]
37.627742
37.625631
[37.622219, 37.622467]
37.622467
37.622219
[37.622467, 37.622445]
37.622445
37.622467
[37.622445, 37.622804]
37.622804
37.622445
[37.622804, 37.622742]
37.622742
37.622804
[37.622742, 37.623037]
37.623037
37.622742
[37.623037, 37.623045]
37.623045
37.623037
[37.623045, 37.625184]
37.625184
37.623045
[37.625184, 37.625286]
37.625286
37.625184
[37.625286, 37.625631]
37.625631
37.625286
[37.6

37.625803
37.625649
[37.625803, 37.625206]
37.625206
37.625803
[37.625206, 37.625342]
37.625342
37.625206
[37.625342, 37.625505]
37.625505
37.625342
[37.625505, 37.627742]
37.627742
37.625505
[37.628317, 37.628306]
37.628306
37.628317
[37.628306, 37.62647]
37.62647
37.628306
[37.62647, 37.625649]
37.625649
37.62647
[37.625649, 37.625375]
37.625375
37.625649
[37.625375, 37.624114]
37.624114
37.625375
[37.624114, 37.624165]
37.624165
37.624114
[37.624165, 37.623889]
37.623889
37.624165
[37.623889, 37.623808]
37.623808
37.623889
[37.623808, 37.622673]
37.622673
37.623808
[37.622673, 37.622251]
37.622251
37.622673
[37.622251, 37.622203]
37.622203
37.622251
[37.629377, 37.629268]
37.629268
37.629377
[37.629268, 37.629265]
37.629265
37.629268
[37.629265, 37.628877]
37.628877
37.629265
[37.628877, 37.628737]
37.628737
37.628877
[37.628737, 37.628731]
37.628731
37.628737
[37.628731, 37.626555]
37.626555
37.628731
[37.626555, 37.625649]
37.625649
37.626555
[37.625649, 37.625803]
37.625803
37.62

37.629794
[37.629773, 37.629512]
37.629512
37.629773
[37.629512, 37.629498]
37.629498
37.629512
[37.629498, 37.629162]
37.629162
37.629498
[37.629162, 37.629155]
37.629155
37.629162
[37.629155, 37.628657]
37.628657
37.629155
[37.628657, 37.627292]
37.627292
37.628657
[37.627292, 37.626555]
37.626555
37.627292
[37.626555, 37.625649]
37.625649
37.626555
[37.625649, 37.625375]
37.625375
37.625649
[37.625375, 37.624114]
37.624114
37.625375
[37.624114, 37.624165]
37.624165
37.624114
[37.624165, 37.623808]
37.623808
37.624165
[37.623808, 37.622673]
37.622673
37.623808
[37.622673, 37.622251]
37.622251
37.622673
[37.622251, 37.622203]
37.622203
37.622251
[37.632236, 37.63232]
37.63232
37.632236
[37.63232, 37.631048]
37.631048
37.63232
[37.631048, 37.631035]
37.631035
37.631048
[37.631035, 37.628642]
37.628642
37.631035
[37.628642, 37.627665]
37.627665
37.628642
[37.627665, 37.627699]
37.627699
37.627665
[37.627699, 37.625505]
37.625505
37.627699
[37.625505, 37.625416]
37.625416
37.625505
[37.6

...

37.643237
[37.643086, 37.642848]
37.642848
37.643086
[37.642848, 37.641901]
37.641901
37.642848
[37.641901, 37.641378]
37.641378
37.641901
[37.641378, 37.637119]
37.637119
37.641378
[37.637119, 37.637005]
37.637005
37.637119
[37.637005, 37.636798]
37.636798
37.637005
[37.636798, 37.636783]
37.636783
37.636798
[37.636783, 37.63637]
37.63637
37.636783
[37.63637, 37.636389]
37.636389
37.63637
[37.636389, 37.636026]
37.636026
37.636389
[37.636026, 37.635748]
37.635748
37.636026
[37.635748, 37.635352]
37.635352
37.635748
[37.635352, 37.634381]
37.634381
37.635352
[37.634381, 37.634422]
37.634422
37.634381
[37.634422, 37.634227]
37.634227
37.634422
[37.634227, 37.634234]
37.634234
37.634227
[37.634234, 37.633797]
37.633797
37.634234
[37.633797, 37.633806]
37.633806
37.633797
[37.633806, 37.631048]
37.631048
37.633806
[37.631048, 37.631035]
37.631035
37.631048
[37.631035, 37.627665]
37.627665
37.631035
[37.627665, 37.627699]
37.627699
37.627665
[37.627699, 37.625416]
37.625416
37.627699
[37.6

37.626555
[37.625649, 37.625375]
37.625375
37.625649
[37.625375, 37.624114]
37.624114
37.625375
[37.624114, 37.624165]
37.624165
37.624114
[37.624165, 37.623808]
37.623808
37.624165
[37.623808, 37.622673]
37.622673
37.623808
[37.622673, 37.622251]
37.622251
37.622673
[37.622251, 37.622203]
37.622203
37.622251
[37.634146, 37.632819]
37.632819
37.634146
[37.632819, 37.632252]
37.632252
37.632819
[37.632252, 37.632203]
37.632203
37.632252
[37.632203, 37.631022]
37.631022
37.632203
[37.631022, 37.631096]
37.631096
37.631022
[37.631096, 37.630942]
37.630942
37.631096
[37.630942, 37.629968]
37.629968
37.630942
[37.629968, 37.629794]
37.629794
37.629968
[37.629794, 37.629773]
37.629773
37.629794
[37.629773, 37.629512]
37.629512
37.629773
[37.629512, 37.629498]
37.629498
37.629512
[37.629498, 37.629162]
37.629162
37.629498
[37.629162, 37.629155]
37.629155
37.629162
[37.629155, 37.628657]
37.628657
37.629155
[37.628657, 37.627292]
37.627292
37.628657
[37.627292, 37.626555]
37.626555
37.627292
[

37.63758
[37.637427, 37.636545]
37.636545
37.637427
[37.636545, 37.636512]
37.636512
37.636545
[37.636512, 37.636382]
37.636382
37.636512
[37.636382, 37.636401]
37.636401
37.636382
[37.636401, 37.635313]
37.635313
37.636401
[37.635313, 37.634433]
37.634433
37.635313
[37.634433, 37.634481]
37.634481
37.634433
[37.634481, 37.634368]
37.634368
37.634481
[37.634368, 37.634474]
37.634474
37.634368
[37.634474, 37.629268]
37.629268
37.634474
[37.629268, 37.629265]
37.629265
37.629268
[37.629265, 37.628877]
37.628877
37.629265
[37.628877, 37.628731]
37.628731
37.628877
[37.628731, 37.626555]
37.626555
37.628731
[37.626555, 37.625649]
37.625649
37.626555
[37.625649, 37.625803]
37.625803
37.625649
[37.625803, 37.625206]
37.625206
37.625803
[37.625206, 37.625342]
37.625342
37.625206
[37.625342, 37.625505]
37.625505
37.625342
[37.625505, 37.626241]
37.626241
37.625505
[37.637803, 37.63758]
37.63758
37.637803
[37.63758, 37.637427]
37.637427
37.63758
[37.637427, 37.636545]
37.636545
37.637427
[37.63

37.625342
[37.625505, 37.627742]
37.627742
37.625505
[37.642599, 37.642504]
37.642504
37.642599
[37.642504, 37.643328]
37.643328
37.642504
[37.643328, 37.645501]
37.645501
37.643328
[37.645501, 37.646101]
37.646101
37.645501
[37.646101, 37.644494]
37.644494
37.646101
[37.644494, 37.640169]
37.640169
37.644494
[37.640169, 37.640213]
37.640213
37.640169
[37.640213, 37.639897]
37.639897
37.640213
[37.639897, 37.639741]
37.639741
37.639897
[37.639741, 37.638251]
37.638251
37.639741
[37.638251, 37.636775]
37.636775
37.638251
[37.636775, 37.630041]
37.630041
37.636775
[37.630041, 37.629866]
37.629866
37.630041
[37.629866, 37.629637]
37.629637
37.629866
[37.629637, 37.625742]
37.625742
37.629637
[37.625742, 37.624702]
37.624702
37.625742
[37.624702, 37.624719]
37.624719
37.624702
[37.624719, 37.623787]
37.623787
37.624719
[37.623787, 37.623808]
37.623808
37.623787
[37.623808, 37.622673]
37.622673
37.623808
[37.622673, 37.622251]
37.622251
37.622673
[37.622251, 37.622203]
37.622203
37.622251
[

37.635997
37.636586
[37.635997, 37.636139]
37.636139
37.635997
[37.636139, 37.635764]
37.635764
37.636139
[37.635764, 37.63354]
37.63354
37.635764
[37.63354, 37.629641]
37.629641
37.63354
[37.629641, 37.629803]
37.629803
37.629641
[37.629803, 37.626749]
37.626749
37.629803
[37.626749, 37.626724]
37.626724
37.626749
[37.626724, 37.625681]
37.625681
37.626724
[37.625681, 37.625803]
37.625803
37.625681
[37.625803, 37.625206]
37.625206
37.625803
[37.625206, 37.625342]
37.625342
37.625206
[37.625342, 37.625505]
37.625505
37.625342
[37.625505, 37.627742]
37.627742
37.625505
[37.636757, 37.636586]
37.636586
37.636757
[37.636586, 37.635997]
37.635997
37.636586
[37.635997, 37.636139]
37.636139
37.635997
[37.636139, 37.635764]
37.635764
37.636139
[37.635764, 37.63354]
37.63354
37.635764
[37.63354, 37.629641]
37.629641
37.63354
[37.629641, 37.629803]
37.629803
37.629641
[37.629803, 37.626749]
37.626749
37.629803
[37.626749, 37.626724]
37.626724
37.626749
[37.626724, 37.625681]
37.625681
37.626724

In [196]:
c

Counter({37.616985: 4,
         37.617231: 3,
         37.617652: 4,
         37.617593: 4,
         37.621781: 6,
         37.621735: 2,
         37.622208: 2,
         37.622203: 131,
         37.61767: 2,
         37.61959: 2,
         37.619634: 2,
         37.620018: 2,
         37.620199: 2,
         37.622527: 24,
         37.622698: 18,
         37.627108: 42,
         37.62722: 62,
         37.627874: 62,
         37.627869: 48,
         37.627742: 131,
         37.62222: 10,
         37.622251: 236,
         37.622673: 236,
         37.623037: 80,
         37.625198: 64,
         37.625286: 102,
         37.625631: 102,
         37.626241: 131,
         37.632121: 8,
         37.632089: 6,
         37.631199: 8,
         37.631109: 8,
         37.630697: 8,
         37.630574: 36,
         37.629836: 36,
         37.628587: 36,
         37.628524: 36,
         37.627884: 39,
         37.62795: 30,
         37.627628: 46,
         37.627679: 6,
         37.627282: 6,
         

In [192]:
all_routes

[[37.617231, 37.616985],
 [37.616985, 37.617652],
 [37.617652, 37.617593],
 [37.617593, 37.621781],
 [37.621781, 37.621735],
 [37.621735, 37.622208],
 [37.622208, 37.622203],
 [37.617231, 37.61767],
 [37.61767, 37.61959],
 [37.61959, 37.619634],
 [37.619634, 37.620018],
 [37.620018, 37.620199],
 [37.620199, 37.622527],
 [37.622527, 37.622698],
 [37.622698, 37.627108],
 [37.627108, 37.62722],
 [37.62722, 37.627874],
 [37.627874, 37.627869],
 [37.627869, 37.627742],
 [37.617231, 37.616985],
 [37.616985, 37.617652],
 [37.617652, 37.617593],
 [37.617593, 37.62222],
 [37.62222, 37.622251],
 [37.622251, 37.622673],
 [37.622673, 37.623037],
 [37.623037, 37.625198],
 [37.625198, 37.625286],
 [37.625286, 37.625631],
 [37.625631, 37.626241],
 [37.632089, 37.632121],
 [37.632121, 37.631199],
 [37.631199, 37.631109],
 [37.631109, 37.630697],
 [37.630697, 37.630574],
 [37.630574, 37.629836],
 [37.629836, 37.628587],
 [37.628587, 37.628524],
 [37.628524, 37.627884],
 [37.627884, 37.62795],
 [37.6279

In [191]:
Counter([r[1] for r in all_routes()])

TypeError: 'list' object is not callable

In [190]:
all_routes

[[37.617231, 37.616985],
 [37.616985, 37.617652],
 [37.617652, 37.617593],
 [37.617593, 37.621781],
 [37.621781, 37.621735],
 [37.621735, 37.622208],
 [37.622208, 37.622203],
 [37.617231, 37.61767],
 [37.61767, 37.61959],
 [37.61959, 37.619634],
 [37.619634, 37.620018],
 [37.620018, 37.620199],
 [37.620199, 37.622527],
 [37.622527, 37.622698],
 [37.622698, 37.627108],
 [37.627108, 37.62722],
 [37.62722, 37.627874],
 [37.627874, 37.627869],
 [37.627869, 37.627742],
 [37.617231, 37.616985],
 [37.616985, 37.617652],
 [37.617652, 37.617593],
 [37.617593, 37.62222],
 [37.62222, 37.622251],
 [37.622251, 37.622673],
 [37.622673, 37.623037],
 [37.623037, 37.625198],
 [37.625198, 37.625286],
 [37.625286, 37.625631],
 [37.625631, 37.626241],
 [37.632089, 37.632121],
 [37.632121, 37.631199],
 [37.631199, 37.631109],
 [37.631109, 37.630697],
 [37.630697, 37.630574],
 [37.630574, 37.629836],
 [37.629836, 37.628587],
 [37.628587, 37.628524],
 [37.628524, 37.627884],
 [37.627884, 37.62795],
 [37.6279

In [188]:
Counter(all_routes)

TypeError: unhashable type: 'list'

In [146]:
type(all_routes[0][0])

str

In [132]:
all_routes[0]

[[37.617231, 55.72235], [37.616985, 55.723148]]

In [122]:
from collections import Counter

In [130]:
type(seq[0][0])

str

In [147]:
from collections import Counter
seq = [['a','b','a','c'], ['a','b','c','d']]
Counter(x for xs in seq for x in set(xs))

Counter({'c': 2, 'b': 2, 'a': 2, 'd': 1})

In [167]:
Counter(all_routes[0])

TypeError: unhashable type: 'list'

In [186]:
tmp = Counter(x for xs in all_routes for x in set(xs))

In [187]:
tmp

Counter({37.616985: 4,
         37.617231: 3,
         37.617652: 4,
         37.617593: 4,
         37.621781: 6,
         37.621735: 2,
         37.622208: 2,
         37.622203: 131,
         37.61767: 2,
         37.61959: 2,
         37.619634: 2,
         37.620018: 2,
         37.620199: 2,
         37.622527: 24,
         37.622698: 18,
         37.627108: 42,
         37.62722: 62,
         37.627874: 62,
         37.627869: 48,
         37.627742: 131,
         37.62222: 10,
         37.622251: 236,
         37.622673: 236,
         37.623037: 80,
         37.625198: 64,
         37.625286: 102,
         37.625631: 102,
         37.626241: 131,
         37.632121: 8,
         37.632089: 6,
         37.631199: 8,
         37.631109: 8,
         37.630697: 8,
         37.630574: 36,
         37.629836: 36,
         37.628587: 36,
         37.628524: 36,
         37.627884: 39,
         37.62795: 30,
         37.627628: 46,
         37.627679: 6,
         37.627282: 6,
         

In [170]:
tmp

Counter({'[37.616985, 55.723148]': 4,
         '[37.617231, 55.72235]': 3,
         '[37.617652, 55.723202]': 4,
         '[37.617593, 55.723424]': 4,
         '[37.621781, 55.72369]': 6,
         '[37.621735, 55.723023]': 2,
         '[37.622208, 55.723019]': 2,
         '[37.622203, 55.722637]': 131,
         '[37.61767, 55.720968]': 2,
         '[37.61959, 55.721207]': 2,
         '[37.619634, 55.72104]': 2,
         '[37.620018, 55.721093]': 2,
         '[37.620199, 55.720688]': 2,
         '[37.622527, 55.721023]': 24,
         '[37.622698, 55.721007]': 18,
         '[37.627108, 55.721367]': 42,
         '[37.62722, 55.721465]': 62,
         '[37.627874, 55.721463]': 62,
         '[37.627869, 55.721757]': 48,
         '[37.627742, 55.72211]': 131,
         '[37.62222, 55.723707]': 10,
         '[37.622251, 55.723947]': 236,
         '[37.622673, 55.723922]': 236,
         '[37.623037, 55.72584]': 64,
         '[37.625198, 55.726287]': 64,
         '[37.625286, 55.726128]': 102,
  

In [154]:
tmp.most_common()

[('[37.625649, 55.730064]', 330),
 ('[37.625505, 55.726659]', 330),
 ('[37.625206, 55.727647]', 304),
 ('[37.625342, 55.726962]', 304),
 ('[37.626555, 55.731733]', 288),
 ('[37.622251, 55.723947]', 236),
 ('[37.622673, 55.723922]', 236),
 ('[37.625803, 55.72961]', 214),
 ('[37.627292, 55.732519]', 204),
 ('[37.623808, 55.729219]', 154),
 ('[37.624165, 55.729739]', 138),
 ('[37.625375, 55.730082]', 136),
 ('[37.624114, 55.730023]', 132),
 ('[37.622203, 55.722637]', 131),
 ('[37.627742, 55.72211]', 131),
 ('[37.626241, 55.724996]', 131),
 ('[37.625286, 55.726128]', 102),
 ('[37.625631, 55.726186]', 102),
 ('[37.625416, 55.726594]', 88),
 ('[37.629641, 55.73052]', 88),
 ('[37.629803, 55.730415]', 88),
 ('[37.626749, 55.730126]', 88),
 ('[37.626724, 55.7302]', 88),
 ('[37.625681, 55.730124]', 88),
 ('[37.623889, 55.729608]', 84),
 ('[37.63354, 55.730928]', 76),
 ('[37.629265, 55.731887]', 66),
 ('[37.628877, 55.73191]', 66),
 ('[37.628731, 55.731594]', 66),
 ('[37.623037, 55.72584]', 64),


In [101]:
routesDubl.loc[0]['route']['coordinates']

[[37.617231, 55.72235],
 [37.61767, 55.720968],
 [37.61959, 55.721207],
 [37.619634, 55.72104],
 [37.620018, 55.721093],
 [37.620199, 55.720688],
 [37.622527, 55.721023],
 [37.622698, 55.721007],
 [37.627108, 55.721367],
 [37.62722, 55.721465],
 [37.627874, 55.721463],
 [37.627869, 55.721757],
 [37.627742, 55.72211]]